In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import timeit
import urllib.request  
import glob
import os
import shutil
import urllib.request
import requests
import urllib
from urllib.request import urlopen
import urllib3
import multiprocessing as mul
from multiprocessing import Process

In [2]:
cd "/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/"

/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta


In [3]:
df_telescope = pd.read_csv("code/Telescope_info_GCNs.txt", sep="\t")
df_telescope['GRB'] = df_telescope['GRB'].astype('string')
df_telescope['GCN'] = df_telescope['GCN'].astype('string')
df_telescope

,GRB,GCN,Telescope
0,000131,1133,hubble space WFPC
1,000131,537,"2.34m, 1.02m vainu bappu"
2,000131,26700,Swift/UVOT
3,000131,534,8.2-m antu
4,000301,597,guillermo haro 2.1m
...,...,...,...
4846,991216,468,CTIO/schmidt and CTIO/yale 1m
4847,991216,466,1.8m perkins
4848,991216,465,bao 60/90 cm schmidt
4849,991216,487,lowell/1.8m perkins


In [4]:
cd "/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/"

/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta


In [5]:
path = "/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/"
dir_list = os.listdir(path)
for file in dir_list:
    if not file.endswith('.txt'):
        dir_list.remove(file)
dir_list.remove('aaaaaaa.ipynb')
for file in dir_list:
    df=pd.read_csv(file,sep='\s+')
    #df.columns = df.columns.str.replace('gcn', 'Source').replace('GCN', 'Source').replace()
    

file = '100302A-has-date-time.tx'
df=pd.read_csv(file,sep='\s+')
delta = []
from datetime import datetime
start_time = df['Time'][0]
for i in range(len(df)):
    end_time = df['Time'][i]
    t1 = datetime.strptime(start_time, "%H:%M:%S.%f")
    #print('Start time:', t1.time())

    t2 = datetime.strptime(end_time, "%H:%M:%S.%f")
    #print('End time:', t2.time())
    temp = t2 - t1
    delta.append(temp.total_seconds() + (float(df['exp'][i])/2.0))
delta[-1] += 86400
df.drop(['date','Time','exp'],axis=1,inplace=True)
df.insert(loc=0,column='Time',value=delta)
df.to_csv(file, sep='\t')

In [7]:
path = "/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/"
dir_list = os.listdir(path)
for file in dir_list:
    if not file.endswith('.txt'):
        dir_list.remove(file)
for file in dir_list:
    grb=file.split('.')[0]
    if grb[0]=='0':
        grb = grb.replace('A','',1)
    df=pd.read_csv(file,sep='\s+')
    if 'Source' in df:        
        
        df['Source'] = df['Source'].astype('string')
        if not('Telescope' in df):    
            df.insert(loc=4, column='Telescope', value='none')
            df['Telescope'] = df['Telescope'].astype('string')
            df_grb=pd.DataFrame()
            for j in range(len(df_telescope)):
                if df_telescope['GRB'][j]==grb:
                    df_grb=df_telescope[df_telescope.GRB.isin([grb])]
            index=np.array(df_grb.index)
            for i in range(len(df)):
                for j in index:
                    if df['Source'][i]==df_grb['GCN'][j]:
                        df['Telescope'][i]=df_grb['Telescope'][j]
            df.to_csv(file, sep='\t',index=False)

        else:
            print(file + " already has telescope")
            #files_with_telescope.append(file)

050401A_Zaninoni_mag.txt already has telescope
050525A_Zaninoni_mag.txt already has telescope
050408A_Zaninoni_mag.txt already has telescope
050319A_Zaninoni_mag.txt already has telescope
050803.txt already has telescope
050814.txt already has telescope
100302A.txt already has telescope


ParserError: Error tokenizing data. C error: Expected 2 fields in line 145, saw 16


In [19]:
path = "/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/"
dir_list = os.listdir(path + 'gcn3')
file_url = 'gcn3'
for file in dir_list:
    if not file.endswith('.txt'):
        dir_list.remove(file)


def sys_cola(dir_list):
    completed = []
    for file in dir_list:
        df=pd.read_csv(file,sep='\s+')
        sysval = []
        if 'System' in df:
                df.drop(['System'],axis=1,inplace=True)
        else:
            if 'Source' in df:
            
                for i in range(len(df)):
                    try:
                        furl = file_url.replace('gcname',str(df['Source'][i]))
                        ifile = open(furl)
                        temp = ifile.read()
                        if '2MASS' in temp and not('USNO' in temp):
                            sysval.append('Johnson')
                        elif 'USNO' in temp and not('2MASS' in temp):
                            sysval.append('Vega')
                        elif 'SDSS' in temp or 'Skymapper' in temp or 'Sloan Digital Sky Survey' in temp:
                            sysval.append('AB')
                        elif 'Keck' in temp:
                            sysval.append('Johnson')
                        elif 'Swift/UVOT' in temp:
                            sysval.append('Swift')
                        else: sysval.append('?')
                        df.insert(loc=4,column='System',value=sysval)
                        df.to_csv(file,sep='\t')
                    except:
                        continue
                df.insert(loc=4,column='System',value=sysval)
                df.drop(['index'],axis=1,inplace=True)
        completed.append(file.replace('/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/',''))
    return completed


def sys_col(dir_list,var):
    var.put(sys_cola(dir_list))
                
# for line in urllib.request.urlopen(file_url):
    #print(line.decode('utf-8')) 

In [ ]:
output3=[]
for i in range(10):
    var1 = mul.Queue()
    var2 = mul.Queue()
    var3 = mul.Queue()
    var4 = mul.Queue()
    var5 = mul.Queue()
    var6 = mul.Queue()
    var7 = mul.Queue()
    var8 = mul.Queue()
    var9 = mul.Queue()
    var10 = mul.Queue()
    var11 = mul.Queue()
    var12 = mul.Queue()
    p1 = Process(target=sys_col, args=(dir_list[:9], var1))
    p2 = Process(target=sys_col, args=(dir_list[9:18], var2))
    p3 = Process(target=sys_col, args=(dir_list[18:27], var3))
    p4 = Process(target=sys_col, args=(dir_list[27:36], var4))
    p5 = Process(target=sys_col, args=(dir_list[36:45], var5))
    p6 = Process(target=sys_col, args=(dir_list[45:54], var6))
    p7 = Process(target=sys_col, args=(dir_list[54:63], var7))
    p8 = Process(target=sys_col, args=(dir_list[63:72], var8))
    p9 = Process(target=sys_col, args=(dir_list[72:81], var9))
    p10 = Process(target=sys_col, args=(dir_list[81:90], var10))
    p11 = Process(target=sys_col, args=(dir_list[90:99], var11))
    p12 = Process(target=sys_col, args=(dir_list[99:108], var12))
    p1.start()
    p2.start()
    p3.start()
    p4.start()
    p5.start()
    p6.start()
    p7.start()
    p8.start()
    p9.start()
    p10.start()
    p11.start()
    p12.start()
    val1 = var1.get()
    val2 = var2.get()
    val3 = var3.get()
    val4 = var4.get()
    val5 = var5.get()
    val6 = var6.get()
    val7 = var7.get()
    val8 = var8.get()
    val9 = var9.get()
    val10 = var10.get()
    val11 = var11.get()
    val12 = var12.get()
    p1.join()
    p2.join()
    p3.join()
    p4.join()
    p5.join()
    p6.join()
    p7.join()
    p8.join()
    p9.join()
    p10.join()
    p11.join()
    p12.join()
    
    p1.close()
    p2.close()
    p3.close()
    p4.close()
    p5.close()
    p6.close()
    p7.close()
    p8.close()
    p9.close()
    p10.close()
    p11.close()
    p12.close()
    output3.append([val1, val2, val3, val4, val5, val6, val7, val8, val9, val10, val11, val12])